In [1]:
# default_exp datasets

In [2]:
# export

import numpy as np
import pandas as pd

def events_metadata(events, trial_query='subject > -1'):

    # build list lengths argument, careful to avoid including filtered-out list lengths
    if 'list length' in events.columns:
        list_lengths = [int(each) for each in pd.unique(events.query(trial_query)["list length"])]
        if trial_query:
            trial_query += ' & `list length` == {list_length}'
        else:
            trial_query = '`list length` == {list_length}'
    else:
        list_lengths = [int(np.max(events.query(trial_query).input))]
    
    # build trials argument, careful to filter out recalls and presentations rows with same query
    trials = []
    presentations = []
    trial_details = events.pivot_table(index=['subject', 'list'], dropna=False)
    for list_length in list_lengths:
        trial_filter = trial_details.eval(trial_query.format(list_length=list_length))
        trials_df = events.pivot_table(index=['subject', 'list'], columns='output', values='item', dropna=False)
        trials_array = trials_df.to_numpy(na_value=0).astype('int32')[trial_filter]
        #trials_array = np.hstack((trials_array, np.zeros((trials_array.shape[0], 1), dtype=np.int32)))
        trials.append(trials_array)

        presentations_df = events.pivot_table(index=['subject', 'list'], columns='input', values='item', dropna=False)
        presentations_array = presentations_df.to_numpy(na_value=0).astype('int32')[trial_filter]
        presentations.append(presentations_array)
    
    # dont make trials or lists_lengths or presentations lists if they are only one element
    if len(trials) == 1:
        trials = trials[0]
    if len(list_lengths) == 1:
        list_lengths = list_lengths[0]
    if len(presentations) == 1:
        presentations = presentations[0]

    return trials, list_lengths, presentations, trial_details[trial_filter].reset_index()

In [3]:
events = pd.read_csv('../../data/PEERS.csv')

events

,subject,list,item,input,output,study,recall,repeat,intrusion,item_string_index,item_string,session,session_list,task
0,0,0,1,1.0,6.0,True,True,0,False,1062,PIE,0,1,-1
1,0,0,2,2.0,7.0,True,True,0,False,219,CAKE,0,1,-1
2,0,0,3,3.0,NaN,True,False,0,False,779,KITE,0,1,-1
3,0,0,4,4.0,NaN,True,False,0,False,148,BONE,0,1,-1
4,0,0,5,5.0,NaN,True,False,0,False,668,GUARDIAN,0,1,-1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
570321,151,110,12,12.0,4.0,True,True,0,False,36,ARCHITECT,16,15,1
570322,151,110,13,13.0,NaN,True,False,0,False,188,BUGGY,16,15,1
570323,151,110,14,14.0,NaN,True,False,0,False,992,OWL,16,15,1
570324,151,110,15,15.0,1.0,True,True,0,False,1458,THIMBLE,16,15,1


In [4]:
trials, list_lengths, presentations, trial_details = events_metadata(events)

np.shape(trials), np.shape(presentations), np.shape(trial_details)

((44232, 16), (44232, 16), (44232, 11))

In [5]:
trial_details

input  intrusion  item  item_string_index  output  recall  \
subject list                                                              
0       0       8.5        0.0   8.5           922.9375     4.0  0.4375   
        1       8.5        0.0   8.5           933.6875     3.0  0.3125   
        2       8.5        0.0   8.5           798.8125     4.5  0.5000   
        3       8.5        0.0   8.5           707.4375     3.5  0.3750   
        4       8.5        0.0   8.5           858.5000     5.0  0.5625   
...             ...        ...   ...                ...     ...     ...   
151     288     NaN        NaN   NaN                NaN     NaN     NaN   
        289     NaN        NaN   NaN                NaN     NaN     NaN   
        290     NaN        NaN   NaN                NaN     NaN     NaN   
        291     NaN        NaN   NaN                NaN     NaN     NaN   
        292     NaN        NaN   NaN                NaN     NaN     NaN   

              repeat  session  session_list  study  task  
subject list                                              
0       0        0.0      0.0           1.0    1.0  -1.0  
        1        0.0      0.0           2.0    1.0  -1.0  
        2        0.0      0.0           3.0    1.0  -1.0  
        3        0.0      0.0           4.0    1.0  -1.0  
        4        0.0      0.0           5.0    1.0   1.0  
...              ...      ...           ...    ...   ...  
151     288      NaN      NaN           NaN    NaN   NaN  
        289      NaN      NaN           NaN    NaN   NaN  
        290      NaN      NaN           NaN    NaN   NaN  
        291      NaN      NaN           NaN    NaN   NaN  
        292      NaN      NaN           NaN    NaN   NaN  

[44232 rows x 11 columns]

In [6]:
trials, list_lengths, presentations, trial_details = events_metadata(events, 'session != 0')

np.shape(trials), np.shape(presentations), np.shape(trial_details)

((41952, 16), (41952, 16), (41952, 11))

In [7]:
trial_details

input  intrusion      item  item_string_index  output    recall  \
subject list                                                                    
0       16      8.5   0.200000  8.150000         828.500000     4.0  0.350000   
        17      8.5   0.058824  8.117647         873.176471     2.5  0.235294   
        18      8.5   0.157895  7.684211         863.631579     4.5  0.421053   
        19      8.5   0.000000  8.500000         718.687500     3.5  0.375000   
        20      8.5   0.000000  8.500000         663.625000     5.0  0.562500   
...             ...        ...       ...                ...     ...       ...   
151     288     NaN        NaN       NaN                NaN     NaN       NaN   
        289     NaN        NaN       NaN                NaN     NaN       NaN   
        290     NaN        NaN       NaN                NaN     NaN       NaN   
        291     NaN        NaN       NaN                NaN     NaN       NaN   
        292     NaN        NaN       NaN                NaN     NaN       NaN   

              repeat  session  session_list     study      task  
subject list                                                     
0       16       0.0      1.0           1.0  0.800000  0.400000  
        17       0.0      1.0           2.0  0.941176  0.529412  
        18       0.0      1.0           3.0  0.842105  0.421053  
        19       0.0      1.0           4.0  1.000000  1.000000  
        20       0.0      1.0           5.0  1.000000  0.000000  
...              ...      ...           ...       ...       ...  
151     288      NaN      NaN           NaN       NaN       NaN  
        289      NaN      NaN           NaN       NaN       NaN  
        290      NaN      NaN           NaN       NaN       NaN  
        291      NaN      NaN           NaN       NaN       NaN  
        292      NaN      NaN           NaN       NaN       NaN  

[41952 rows x 11 columns]